In [ ]:
# Cell 8: render SHAP summary for misclassified examples with reproducible randomness and robust fallbacks
import numpy as np
import shap
import matplotlib.pyplot as plt
import pandas as pd
import warnings
import inspect

# Preconditions (variables that must exist in the session):
# - sm                : normalized SHAP array with shape (n_samples, n_features)
# - X_mis_trans       : transformed numeric array for misclassified examples
# - X_train_trans     : transformed numeric array for training set
# - trans_feature_names: list of transformed feature names (len == n_features)

plot_X = X_mis_trans
feature_names = trans_feature_names if 'trans_feature_names' in globals() else [f"f{i}" for i in range(sm.shape[1])]

# Create a reproducible RNG using the NumPy Generator API
rng = np.random.default_rng(0)

# Detect whether the installed SHAP accepts an explicit rng parameter
accepts_rng = False
try:
    sig = inspect.signature(shap.summary_plot)
    accepts_rng = 'rng' in sig.parameters
except Exception:
    accepts_rng = False

if accepts_rng:
    # Preferred: pass an explicit RNG to make randomness explicit and silence future warnings
    shap.summary_plot(sm, plot_X, feature_names=feature_names, show=True, rng=rng)
else:
    # Attempt to call with rng (may raise TypeError on older SHAP versions)
    try:
        shap.summary_plot(sm, plot_X, feature_names=feature_names, show=True, rng=rng)
    except TypeError:
        # Fallback for older SHAP: seed the legacy global RNG and suppress the FutureWarning
        np.random.seed(0)
        with warnings.catch_warnings():
            warnings.filterwarnings("ignore", category=FutureWarning)
            #Learners to type
            

# If interactive rendering fails for other reasons, draw a static mean-|SHAP| bar chart
try:
    plt.gcf().canvas.draw_idle()
except Exception:
    mean_abs = pd.Series(np.abs(sm).mean(axis=0), index=feature_names).sort_values(ascending=True)
    n_display = min(30, len(mean_abs))
    mean_abs.tail(n_display).plot(kind="barh", figsize=(10, max(4, 0.25 * n_display)))
    plt.title("Mean absolute SHAP (fallback)")
    plt.xlabel("mean |SHAP value|")
    plt.tight_layout()
    plt.show()

# Diagnostic prints for reviewer verification
print("sm.shape, X_mis_trans.shape, len(trans_feature_names):", sm.shape, X_mis_trans.shape, len(trans_feature_names))
print("Top features shown:", trans_feature_names[:15])
